<a href="https://colab.research.google.com/github/Tonge-Shim/1/blob/master/smote%2Brfe%2Bforest_clf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [199]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from pandas import DataFrame
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier
sgd_clf=SGDClassifier(max_iter=5, random_state=42)

warnings.filterwarnings(action='ignore')
%matplotlib inline

In [200]:
df= pd.read_csv('samp_cst_feat.csv')
dtype = pd.read_excel('variable_dtype.xlsx')
dtype_categ = dtype[(dtype['dType']=='categorical')]
dtype_numrc = dtype[(dtype['dType']=='numerical')]
numrc_var = dtype_numrc['Variable_Name']
categ_var = dtype_categ['Variable_Name']
numrc_var = numrc_var.values.tolist()
categ_var = categ_var.values.tolist()#0,1
x=df.drop(categ_var, axis=1).drop(['cst_id_di'],axis=1)#10124개
y=pd.read_csv('samp_train.csv').drop(['cst_id_di'],axis=1)#10124
y_nozero= y[(y['MRC_ID_DI']!=0)]#not 0!!!1924
#y_nozeroindex=y_nozero.loc[1:10119]
y_nozeroindexlist=y_nozero.index.tolist()
x_nozero=x.loc[y_nozeroindexlist]

#rfe-feature selection

In [201]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

logreg=LogisticRegression()
selector=RFE(logreg)
selector=selector.fit(x_nozero,y_nozero)

In [202]:
selected_features=DataFrame({'Feature': list(x_nozero.columns), 'Ranking': selector.ranking_})
selected_features.sort_values(by='Ranking').head(100)

,Feature,Ranking
197,VAR227,1
88,VAR105,1
87,VAR104,1
86,VAR103,1
85,VAR102,1
...,...,...
177,VAR206,1
178,VAR207,1
179,VAR209,1
40,VAR046,1


In [203]:
newx=selected_features.sort_values(by='Ranking')['Feature']
newx=newx.loc[197:40]#ranking1stround
newx.tolist()
x_nozero=x_nozero[newx]


#0을 제외한 1924개

In [158]:
x_train, x_test, y_train, y_test=x_nozeroarray[:1400], x_nozeroarray[1400:], y_nozeroarray[:1400], y_nozeroarray[1400:]
shuffle_index=np.random.permutation(1400)
x_train, y_train=x_train[shuffle_index], y_train[shuffle_index]

#0제외한 1924데이터 분류

In [204]:
y_1=y[(y['MRC_ID_DI']==1)]
y_2=y[(y['MRC_ID_DI']==2)]
y_3=y[(y['MRC_ID_DI']==3)]
y_4=y[(y['MRC_ID_DI']==4)]
y_5=y[(y['MRC_ID_DI']==5)]
y_6=y[(y['MRC_ID_DI']==6)]
y_7=y[(y['MRC_ID_DI']==7)]
y_8=y[(y['MRC_ID_DI']==8)]
y_9=y[(y['MRC_ID_DI']==9)]
y_10=y[(y['MRC_ID_DI']==10)]

x_1=x_nozero.loc[y_1.index.tolist()]
x_2=x_nozero.loc[y_2.index.tolist()]
x_3=x_nozero.loc[y_3.index.tolist()]
x_4=x_nozero.loc[y_4.index.tolist()]
x_5=x_nozero.loc[y_5.index.tolist()]
x_6=x_nozero.loc[y_6.index.tolist()]
x_7=x_nozero.loc[y_7.index.tolist()]
x_8=x_nozero.loc[y_8.index.tolist()]
x_9=x_nozero.loc[y_9.index.tolist()]
x_10=x_nozero.loc[y_10.index.tolist()]

#2,3,4,9,10 약 60%정확도

In [212]:
x_210=x_nozero.loc[y_2.index.tolist()+y_3.index.tolist()+y_4.index.tolist()+y_9.index.tolist()+y_10.index.tolist()]
y_210=y.loc[x_210.index.tolist()]
print(x_210.shape)
print(y_210.shape)
#x_210=x_210.to_numpy()
#y_210=y_210.to_numpy()

#smote applied]
from imblearn.over_sampling import SMOTE
sm = SMOTE(ratio='auto', kind='regular')
x_210, y_210 = sm.fit_sample(x_210,y_210)


x_210train, x_210test, y_210train, y_210test=x_210[:240], x_210[240:], y_210[:240], y_210[240:]
shuffle_index=np.random.permutation(240)
x_210train, y_210train=x_210train[shuffle_index], y_210train[shuffle_index]
some_digit1=x_210[3]#y_210[3]=2

(144, 99)
(144, 1)


In [213]:
#randomforest
forest_clf=RandomForestClassifier(random_state=42)
forest_clf.fit(x_210train, y_210train)
forest_clf.predict([some_digit1])

array([2])

In [214]:
cross_val_score(forest_clf, x_210train, y_210train, cv=3, scoring="accuracy")


array([0.7375, 0.7   , 0.7625])

In [215]:
#sgd_clf
sgd_clf.fit(x_210train, y_210train)
sgd_clf.predict([some_digit1])
cross_val_score(sgd_clf, x_210train, y_210train, cv=3, scoring="accuracy")


array([0.5625, 0.5625, 0.675 ])

smote는 어떻게 쓰는 지 잘 모르겟다...